# Part 6 - Opportunity Provider - Issuer Mode sub wallet management

## 1. Initialise a controller for Issuer

In [2]:
%autoawait
import time
import asyncio
from termcolor import colored,cprint

from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://yoma-multi-tenant-agent:8021"

# JWT KEY for OP 1 Sub Wallet
%store -r op1_token

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, jwt_token=op1_token)
    

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fc9040eb9a0>


IPython autoawait is `on`, and set to use `asyncio`


## 2. Register Listeners

The handler should get called every time the controller receives a webhook with the topic issue_credential, printing out the payload. The agent calls to this webhook every time it receives an issue-credential protocol message from a credential.

In [3]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Offering: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}

def connections_handler(payload):
    global STATE
    connection_id = payload["connection_id"]
    print("Connection message", payload, connection_id)
    STATE = payload['state']
    if STATE == 'active':
#         print('Connection {0} changed state to active'.format(connection_id))
        print(colored("Connection {0} changed state to active".format(connection_id), "red", attrs=["bold"]))


connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

agent_controller.register_listeners([cred_listener,connection_listener], defaults=True)


Task exception was never retrieved
future: <Task finished name='Task-1' coro=<AriesAgentController.listen_webhooks() done, defined at /aries_basic_controller/aries_controller.py:140> exception=OSError(98, "error while attempting to bind on address ('0.0.0.0', 8022): address already in use")>
Traceback (most recent call last):
  File "/aries_basic_controller/aries_controller.py", line 147, in listen_webhooks
    await self.webhook_site.start()
  File "/opt/conda/lib/python3.8/site-packages/aiohttp/web_runner.py", line 121, in start
    self._server = await loop.create_server(
  File "/opt/conda/lib/python3.8/asyncio/base_events.py", line 1463, in create_server
    raise OSError(err.errno, 'error while attempting '
OSError: [Errno 98] error while attempting to bind on address ('0.0.0.0', 8022): address already in use


## 2. Get current public DID

Before being able to write to any indy based ledger, your agent must have a public DID written on the ledger giving it the authority to write to it. As the cell below shows, this agent does not currently have a public DID. So any writes to the ledger will be rejected.

In [4]:
response = await agent_controller.wallet.get_public_did()
print(response)

{'result': None}


## 8a. Check for any existing connections

In [5]:
# Check for existing connections
connection = await agent_controller.connections.get_connections()
print("EXISTING CONNECTIONS")
for key, value in connection.items():
    for item in value:
        print('ConnectionID:', item['connection_id'], 'Status:',item['state'])

EXISTING CONNECTIONS


## 3. Generate a new DID

Before being able to write a DID to the ledger, you must create one using the wallet api. This api returns the identifier (the DID), and the verification key for that DID. A representation of it's public key. 

In [6]:
# generate new DID
response = await agent_controller.wallet.create_did()

did_object = response['result']
print("New DID", did_object)

New DID {'did': 'C9sv8wheGAmJ2oCFvEENqD', 'verkey': '75a5uRonGPnVbdphDLMFWQeBh1b3H2UQ2NdKR9KETE91', 'posture': 'wallet_only'}


## Write DID to Sovrin Stagingnet

In [8]:
# write new DID to Sovrin Stagingnet
import requests
import json 

url = 'https://selfserve.sovrin.org/nym'

payload = {"network":"stagingnet","did": did_object["did"],"verkey":did_object["verkey"],"paymentaddr":""}

# Adding empty header as parameters are being sent in payload
headers = {}

r = requests.post(url, data=json.dumps(payload), headers=headers)
print(r.json())
print(r.status_code)

{'statusCode': 200, 'headers': {'Access-Control-Allow-Origin': '*'}, 'body': '{"statusCode": 200, "C9sv8wheGAmJ2oCFvEENqD": {"status": "Success", "statusCode": 200, "reason": "Successfully wrote NYM identified by C9sv8wheGAmJ2oCFvEENqD to the ledger with role ENDORSER"}}'}
200


## Accept TAA (Transaction Author Agreement)

In [9]:
response = await agent_controller.ledger.get_taa()
TAA = response['result']['taa_record']
TAA['mechanism'] = "service_agreement"
print(TAA)

{'digest': '8cee5d7a573e4893b08ff53a0761a22a1607df3b3fcd7e75b98696c92879641f', 'text': '\ufeff# Transaction Author Agreement V2\nhttps://sovrin.org/\n\n\n## Summary:\n\n\nThis summary is provided to help you understand your obligations when writing to\nthe Sovrin Ledger Networks-it does not have any legal effect or replace the full\nlegal text of the agreement provided below it.\n\n\n- This agreement grants you permission to write data to the Sovrin Ledger\n  Networks under certain terms and conditions.\n\n\n- You represent and warrant that the data you are writing does not violate any\n  applicable laws or infringe the rights of any other party.\n\n\n- You understand the data you are writing is public and permanent and there can\n  be no guarantee of erasure. This includes public keys and payment addresses.\n\n\n- If it is determined that the data you wrote violated this agreement, the\n  operators of the network can take steps to block it from public access.\n\n\n- The Sovrin Foundat

In [10]:
response = await agent_controller.ledger.accept_taa(TAA)
## Will return {} if successful
print(response)

{}


## 6. Set public DID

Now you are able to assign the DID written to the ledger as public.

In [11]:
response = await agent_controller.wallet.assign_public_did(did_object["did"])
print(response)

{'result': {'did': 'C9sv8wheGAmJ2oCFvEENqD', 'verkey': '75a5uRonGPnVbdphDLMFWQeBh1b3H2UQ2NdKR9KETE91', 'posture': 'public'}}


## 8. Paste the invitation from the [Yoma Agent Issuer notebook](http://localhost:8888/notebooks/Part%204%20-%20Yoma%20Agent%20Credential%20Issuance.ipynb) into the invitation variable here


In [27]:
invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '84159887-f77d-46b3-9500-1d3e442726b9', 'label': 'EXTERNAL', 'recipientKeys': ['AsysiWE8yoLSrnhkuFGjqiBSr46UCiWLU9NnxC7dmuso'], 'serviceEndpoint': 'https://24feee5e7d3a.ngrok.io'}

## 9. Accept the invitation, then move to Alice's notebook

In [28]:
# Receive Invitation
response = await agent_controller.connections.accept_connection(invitation)
# Print out accepted Invite and Alice's connection ID
print("Connection", response)
connection_id = response["connection_id"]
STATE = response["state"]

Connection {'connection_id': 'ac936388-10ba-4515-ae1b-b2d66045a0c7', 'their_role': 'inviter', 'their_label': 'EXTERNAL', 'request_id': '429a01eb-f3f9-4924-820f-0b8093d5d768', 'state': 'request', 'invitation_mode': 'once', 'invitation_key': 'AsysiWE8yoLSrnhkuFGjqiBSr46UCiWLU9NnxC7dmuso', 'accept': 'manual', 'my_did': 'V46MrHw2M8yaJVq8uCbgc3', 'updated_at': '2021-03-11 19:24:04.117996Z', 'created_at': '2021-03-11 19:24:04.058226Z', 'rfc23_state': 'request-sent', 'routing_state': 'none'}


## 13. Check if connection state is active

It should be if you send a trust ping from Alice's side

In [29]:
# Print connection list
connection = await agent_controller.connections.get_connection(connection_id)
print("Opportunity Provider Wallet 1 AGENT CONNECTION")
print(connection)
print("State:", connection["state"])

Opportunity Provider Wallet 1 AGENT CONNECTION
{'connection_id': 'ac936388-10ba-4515-ae1b-b2d66045a0c7', 'their_role': 'inviter', 'their_label': 'EXTERNAL', 'request_id': '429a01eb-f3f9-4924-820f-0b8093d5d768', 'state': 'active', 'invitation_mode': 'once', 'invitation_key': 'AsysiWE8yoLSrnhkuFGjqiBSr46UCiWLU9NnxC7dmuso', 'accept': 'manual', 'my_did': 'V46MrHw2M8yaJVq8uCbgc3', 'updated_at': '2021-03-11 19:24:18.186838Z', 'created_at': '2021-03-11 19:24:04.058226Z', 'rfc23_state': 'completed', 'their_did': 'HgNPGigZAw4DgS7zmCb9E', 'routing_state': 'none'}
State: active


## 10. Check Credential Exchange Records

The agent will have at least one record if you have run through the issuer notebook up until send credential.


In [37]:
response = await agent_controller.issuer.get_records()
results = response["results"]
if len(results) == 0:
    print("You need to first send a credential from the issuer notebook (Alice)")
else:
    cred_record = results[0]
    cred_ex_id = cred_record['credential_exchange_id']
    state = cred_record['state']
    role = cred_record['role']
    attributes = results[0]['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")


Credential exchange 47b97fbb-92a1-4870-826c-33490ad8a90e, role: holder, state: credential_acked
Being offered: [{'name': 'fullname', 'value': 'Lohan Spies'}, {'name': 'mobile', 'value': '0831234567'}, {'name': 'email', 'value': 't@t.com'}]


## 12. Store the credential

Once the issuer has responded to a request by sending the credential, the holder needs to store it to save the credential for later.

First check that the credential record is in the credential_received state


In [22]:
record = await agent_controller.issuer.get_record_by_id(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange 47b97fbb-92a1-4870-826c-33490ad8a90e, role: holder, state: credential_acked


## 3a. Use a previously written schemas


### Youth Member Schema

In [12]:
schema_id = '3pMbbpqEEGzSQkV79jGmDH:2:Youth Member:0.0.1'

### Ecosystem Partner Schema aka Opportunity Provider

In [13]:
schema_id = '3pMbbpqEEGzSQkV79jGmDH:2:Ecosystem Member:0.0.1'

## 3b. (OPTIONAL) Write a Schema to the Ledger

For more details see the [schema-api notebook](http://localhost:8888/notebooks/schema_api.ipynb)

**Note: You will only be able to do this once unless you change the schema_name or version. Once you have a schema written to the ledger it can be reused by multiple entities**

In [14]:
# Define you schema name - must be unique on the ledger
schema_name = "SSI PyDentity Tutorial"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["fullname", "skill", "age"]

response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
schema_id = response["schema_id"]
print(schema_id)


C9sv8wheGAmJ2oCFvEENqD:2:SSI PyDentity Tutorial:0.0.1


In [6]:
# Define you schema name - must be unique on the ledger
schema_name = "Youth Member"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["fullname", "mobile", "email"]

response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
schema_id = response["schema_id"]
print(schema_id)

3pMbbpqEEGzSQkV79jGmDH:2:Youth Member:0.0.1


## 4. Write a Credential Definition to the Ledger

**Note: Again this can only be done once per issuer, per schema_id.**

In [15]:
response = await agent_controller.definitions.write_cred_def(schema_id)

cred_def_id = response["credential_definition_id"]
print(cred_def_id)

C9sv8wheGAmJ2oCFvEENqD:3:CL:194335:default


**Note: You should be able to see both schema and definition transactions on Sovrin Stagingnet network with [Indyscan](https://indyscan.io/home/SOVRIN_STAGINGNET)**

## 5. Populate the Attribues to Issue to Idenity Holder (User)

We will issue a credential to the identity holder consisting of the following attributes:

credential_attributes = [
    {"name": "fullname", "value": name},
    {"name": "skill", "value": "PyDentity SSI Ninja"},
    {"name": "age", "value": age}
]

The notebook will ask you to input the identity holder's full name and age which will be used to populate the schema above with the identity holders attribute information.

In [8]:
name=input("Please enter your name and surname: ")
mobile=input("Please enter your mobile number: ")
email=input("Please enter your email number: ")
credential_attributes = [
    {"name": "fullname", "value": name},
    {"name": "mobile", "value": mobile},
    {"name": "email", "value": email}
]
print(credential_attributes)

Please enter your name and surname: Peter Pan
Please enter your mobile number: 0831234567
Please enter your email number: test@test.com
[{'name': 'fullname', 'value': 'Peter Pan'}, {'name': 'mobile', 'value': '0831234567'}, {'name': 'email', 'value': 'test@test.com'}]


In [ ]:
name=input("Please enter your name and surname: ")
age=input("Please enter your age: ")
credential_attributes = [
    {"name": "fullname", "value": name},
    {"name": "skill", "value": "PyDentity SSI Ninja"},
    {"name": "age", "value": age}
]
print(credential_attributes)

## 7. Download Identity Holder Mobile SSI Wallet
Please download a mobile SSI wallet such as [Connect.me](https://connect.me/), [Trinsic](https://trinsic.id/trinsic-wallet/) or any other Aries compatible wallet.

## 8. Create a connection invitation to scan with the Mobile SSI Wallet

To make a connection with an identity holders wallet we must create a connection invitation and present the QR code to be scanned by the identity holders wallet. The identity holder scan this QR code and then communicate back to the agent through the exposed ngrok endpoint.

Once a connection is in an active state, the agent and the identity holders mobile wallet now have a secure pairwise relationship over which they can exchange verifiable credentials.

## 8a. Check for any existing connections

In [9]:
# Check for existing connections
connection = await agent_controller.connections.get_connections()
print("EXISTING CONNECTIONS")
for key, value in connection.items():
    for item in value:
        print('ConnectionID:', item['connection_id'], 'Status:',item['state'])

EXISTING CONNECTIONS


## 8b. Create an Invitation

In [10]:
# Create Invitation
invite = await agent_controller.connections.create_invitation()
connection_id = invite["connection_id"]
print("Connection ID", connection_id)
print("Invitation")
print(invite['invitation_url'])
inviteURL = invite['invitation_url']

Connection message {'invitation_mode': 'once', 'connection_id': '4c4696cc-f27c-4ba7-9871-e0d9596ffb7e', 'routing_state': 'none', 'invitation_key': 'HChfJyeaiWPKPE3j22eYtx3FtKeeoG8MCEjEsQGY9wdC', 'state': 'invitation', 'rfc23_state': 'invitation-sent', 'updated_at': '2021-03-11 16:09:34.097348Z', 'their_role': 'invitee', 'accept': 'auto', 'created_at': '2021-03-11 16:09:34.097348Z'} 4c4696cc-f27c-4ba7-9871-e0d9596ffb7e
Connection message {'invitation_mode': 'once', 'connection_id': '4c4696cc-f27c-4ba7-9871-e0d9596ffb7e', 'routing_state': 'none', 'invitation_key': 'HChfJyeaiWPKPE3j22eYtx3FtKeeoG8MCEjEsQGY9wdC', 'state': 'invitation', 'rfc23_state': 'invitation-sent', 'updated_at': '2021-03-11 16:09:34.097348Z', 'their_role': 'invitee', 'accept': 'auto', 'created_at': '2021-03-11 16:09:34.097348Z'} 4c4696cc-f27c-4ba7-9871-e0d9596ffb7e
Connection ID 4c4696cc-f27c-4ba7-9871-e0d9596ffb7e
Invitation
https://5e1caa6a25a1.ngrok.io?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3Bl

In [14]:
print(invite['invitation'])

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': 'c241afa6-5ecc-45c8-ac8e-aa1593e6ae25', 'recipientKeys': ['HChfJyeaiWPKPE3j22eYtx3FtKeeoG8MCEjEsQGY9wdC'], 'label': 'EXTERNAL', 'serviceEndpoint': 'https://5e1caa6a25a1.ngrok.io'}


## 8c. Generate QR Code to be scanned with Mobile SSI Wallet

In [ ]:
import qrcode
# Link for connection invitation
input_data = inviteURL
# Creating an instance of qrcode
qr = qrcode.QRCode(
        version=1,
        box_size=10,
        border=5)
qr.add_data(input_data)
qr.make(fit=True)
img = qr.make_image(fill='black', back_color='white')
img.save('issuer_agent_invite_QRcode.png')

from IPython.display import Image
Image(width=400, filename='./issuer_agent_invite_QRcode.png')

## 8d. Check if established connection is in active state

In [15]:
import time

# print('Current state for ConnectionId {} is {}'.format(connection_id,STATE))
print(colored("Current state for ConnectionId {} is {}".format(connection_id,STATE), "magenta", attrs=["bold"]))
while STATE != 'active':
#     print('ConnectionId {0} is not in active state yet'.format(connection_id))
    print(colored("ConnectionId {0} is not in active state yet".format(connection_id), "yellow", attrs=["bold"]))
    trust_ping = await agent_controller.messaging.trust_ping(connection_id,'hello!')
#     print('Trust ping send to ConnectionId {0} to activate connection'.format(trust_ping))
    print(colored("Trust ping send to ConnectionId {0} to activate connection".format(trust_ping), "blue", attrs=["bold"]))
    time.sleep(5)
    
# print('ConnectionId: {0} is now active. Continue with notebook'.format(connection_id))
print(colored("ConnectionId: {0} is now active. Continue with notebook".format(connection_id), "green", attrs=["bold"]))


Current state for ConnectionId 4c4696cc-f27c-4ba7-9871-e0d9596ffb7e is active
ConnectionId: 4c4696cc-f27c-4ba7-9871-e0d9596ffb7e is now active. Continue with notebook


## 9. Send Credential

This sends a credential to a identity holder (User), and automates the rest of the protocol.

There are other ways to issue a credential that require multiple api calls.

**Arguments**
* connection_id: The connection_id of the holder you wish to issue to (MUST be in active state)
* schema_id: The id of the schema you wish to issue
* cred_def_id: The definition (public key) to sign the credential object. This must refer to the schema_id and be written to the ledger by the same public did that is currently being used by the agent.
* attributes: A list of attribute objects as defined above. Must match the schema attributes.
* comment (optional): Any string, defaults to ""
* auto_remove (optional): Boolean, defaults to True. I believe this removes the record of this credential once it has been issued. (TODO: double check)
* trace (optional): Boolean, defaults to False. **Not entirely sure about this one, believe its for logging. Also when set to True it throws an error**

In [16]:
record = await agent_controller.issuer.send_credential(connection_id, schema_id, cred_def_id, credential_attributes, trace=False)
record_id = record['credential_exchange_id']
state = record['state']
role = record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")


Credential exchange 6922e823-88d8-4733-b9c2-ea8e2878e576, role: issuer, state: offer_sent
Handle Credentials
Credential exchange 6922e823-88d8-4733-b9c2-ea8e2878e576, role: issuer, state: offer_sent
Offering: [{'name': 'fullname', 'value': 'Peter Pan'}, {'name': 'mobile', 'value': '0831234567'}, {'name': 'email', 'value': 'test@test.com'}]
Handle Credentials
Credential exchange 6922e823-88d8-4733-b9c2-ea8e2878e576, role: issuer, state: offer_sent
Offering: [{'name': 'fullname', 'value': 'Peter Pan'}, {'name': 'mobile', 'value': '0831234567'}, {'name': 'email', 'value': 'test@test.com'}]
Handle Credentials
Credential exchange 6922e823-88d8-4733-b9c2-ea8e2878e576, role: issuer, state: request_received
Offering: [{'name': 'fullname', 'value': 'Peter Pan'}, {'name': 'mobile', 'value': '0831234567'}, {'name': 'email', 'value': 'test@test.com'}]
Handle Credentials
Credential exchange 6922e823-88d8-4733-b9c2-ea8e2878e576, role: issuer, state: request_received
Offering: [{'name': 'fullname', '

## 10. Accept credential in Mobile SSI Wallet

## 11. End of Tutorial

Be sure to terminate the controller so you can run another tutorial.

In [ ]:
response = await agent_controller.terminate()
print(response)

# Proceed to Part 4 on [Verifier Notebook](http://localhost:8889/notebooks/Part%204%20-%20Verify%20a%20Presentation.ipynb)

Here you will present the attributes issued to you within this tutorial to a verifying entity.